In [49]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras.backend as K
import tensorboard
import shutil

from datetime import datetime as dt

In [50]:
BASE_PATH = "/kaggle/input/osic-pulmonary-fibrosis-progression/"

train_csv = pd.read_csv(BASE_PATH + "train.csv")
test_csv = pd.read_csv(BASE_PATH + "test.csv")

train_csv['Patient_Week'] = train_csv['Patient'].astype(str) + '_' + train_csv['Weeks'].astype(str)

train_data = train_csv.groupby("Patient")
# print(len(train_data))

unique_patients = train_csv['Patient'].unique()
# print(unique_patients)

for patient_id in unique_patients:
    print(train_csv[train_csv['Patient']==patient_id].iloc[0])
    break

Patient             ID00007637202177411956430
Weeks                                      -4
FVC                                      2315
Percent                               58.2536
Age                                        79
Sex                                      Male
SmokingStatus                       Ex-smoker
Patient_Week     ID00007637202177411956430_-4
Name: 0, dtype: object


In [51]:
# Need to add base fvc column, base percentage
train_csv['Weeks'] = train_csv['Weeks'].astype(int)
train_csv['base_week'] = train_csv.groupby('Patient')['Weeks'].transform('min')
train_csv['weeks_passed'] = train_csv['Weeks'] - train_csv['base_week']
train_csv['base_fvc'] = 0
train_csv['base_percent'] = 0

for index in range(len(train_csv)):
    base_data = train_csv[train_csv['Patient']==train_csv.iloc[index]['Patient']].iloc[0]
    train_csv.loc[index, 'base_fvc'] = base_data['FVC']
    train_csv.loc[index, 'base_percent'] = base_data['Percent']
    
train_csv

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,Patient_Week,base_week,weeks_passed,base_fvc,base_percent
0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker,ID00007637202177411956430_-4,-4,0,2315,58.253649
1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker,ID00007637202177411956430_5,-4,9,2315,58.253649
2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker,ID00007637202177411956430_7,-4,11,2315,58.253649
3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker,ID00007637202177411956430_9,-4,13,2315,58.253649
4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker,ID00007637202177411956430_11,-4,15,2315,58.253649
...,...,...,...,...,...,...,...,...,...,...,...,...
1544,ID00426637202313170790466,13,2712,66.594637,73,Male,Never smoked,ID00426637202313170790466_13,0,13,2925,71.824968
1545,ID00426637202313170790466,19,2978,73.126412,73,Male,Never smoked,ID00426637202313170790466_19,0,19,2925,71.824968
1546,ID00426637202313170790466,31,2908,71.407524,73,Male,Never smoked,ID00426637202313170790466_31,0,31,2925,71.824968
1547,ID00426637202313170790466,43,2975,73.052745,73,Male,Never smoked,ID00426637202313170790466_43,0,43,2925,71.824968


In [52]:
# X_data = train_csv[['Sex', 'SmokingStatus']]
X_data = train_csv[['Sex', 'SmokingStatus']]
X_data = pd.get_dummies(data=X_data, drop_first=True)
selected_columns = ['Age', 'weeks_passed', 'base_fvc', 'base_percent']
X_data[selected_columns] = train_csv[selected_columns]
y_data = train_csv['FVC']

X_train = X_data.sample(frac=0.8,random_state=0)
X_test = X_data.drop(X_train.index)

y_train = y_data.sample(frac=0.8,random_state=0).astype(float)
y_test = y_data.drop(y_train.index).astype(float)

train_stats = X_train.describe().transpose()

def norm(x):
  return (x - train_stats['mean']) / train_stats['std']


normalized_X_train = norm(X_train)
normalized_X_test = norm(X_test)

In [53]:
# sns.pairplot(X_train[['Age', 'weeks_passed', 'base_fvc', 'base_percent']], diag_kind="kde")

In [54]:
def laplace_log_likelihood(actual_fvc, predicted_fvc, confidence, return_values = False):
    """
    Calculates the modified Laplace Log Likelihood score for this competition.
    """
    sd_clipped = np.maximum(confidence, 70)
    delta = np.minimum(np.abs(actual_fvc - predicted_fvc), 1000)
    metric = - np.sqrt(2) * delta / sd_clipped - np.log(np.sqrt(2) * sd_clipped)

    if return_values:
        return metric
    else:
        return np.mean(metric)

In [55]:
# def log_custom_acc(y_test, y_pred):
#     sd_clipped = tf.maximum(tf.math.reduce_std(y_pred), 70)
#     delta = tf.minimum(tf.math.abs(tf.math.subtract(y_test, y_pred)), 1000)
    
#     metric = - tf.math.sqrt(2) * delta / sd_clipped - tf.math.log(np.sqrt(2) * sd_clipped)
#     return metric

def score(y_true, y_pred):
    tf.dtypes.cast(y_true, tf.float32)
    tf.dtypes.cast(y_pred, tf.float32)
    sigma = y_pred[:, 2] - y_pred[:, 0]
    fvc_pred = y_pred[:, 1]
    
    #sigma_clip = sigma + C1
    sigma_clip = tf.maximum(sigma, C1)
    delta = tf.abs(y_true[:, 0] - fvc_pred)
    delta = tf.minimum(delta, C2)
    sq2 = tf.sqrt( tf.dtypes.cast(2, dtype=tf.float32) )
    metric = (delta / sigma_clip)*sq2 + tf.math.log(sigma_clip* sq2)
    return K.mean(metric)

In [56]:
# Loss - laplace likelihood

def log_custom_loss(y_true, y_pred):
    tf.dtypes.cast(y_true, tf.float32)
    tf.dtypes.cast(y_pred, tf.float32)
    
    fvc_pred = y_pred
    sigma = tf.math.reduce_std(y_pred)
    
    sigma_clip = tf.maximum(sigma, 70)
    delta = tf.abs(y_true - fvc_pred)
    delta = tf.minimum(delta, 1000)
    sq2 = tf.sqrt( tf.dtypes.cast(2, dtype=tf.float32))
    metric = (delta / sigma_clip) * sq2 + tf.math.log(sigma_clip * sq2)
    return K.mean(metric)

  
def laplace_log_metric(y_true, y_pred):
  return -log_custom_loss(y_true, y_pred)

In [57]:
# For quantile
def tilted_loss(q,y,f):
    e = (y-f)
    return K.mean(K.maximum(q*e, (q-1)*e), axis=-1)

In [58]:
def build_model():
  model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=[len(X_train.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.0005)
#   quantile=0.5
  
#   loss=tf.keras.losses.MeanSquaredLogarithmicError()
#   loss=lambda y,f: tilted_loss(quantile,y,f)
    # loss=log_custom_loss

  model.compile(loss=log_custom_loss,
                optimizer=optimizer,
                metrics=[laplace_log_metric, 'mae', 'mse'])
  return model

model = build_model()

In [59]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 128)               1024      
_________________________________________________________________
dense_11 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 65        
Total params: 9,345
Trainable params: 9,345
Non-trainable params: 0
_________________________________________________________________


In [60]:
# shutil.rmtree('logs')

In [61]:
# %load_ext tensorboard
# %tensorboard --logdir "gdrive/My Drive/kaggle/logs"

In [62]:
EPOCHS = 2500

time_now = dt.now().strftime("%Y%m%d_%H%M%S")
my_callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir=BASE_PATH + 'logs/'+str(time_now)),
]

history = model.fit(
  normalized_X_train, y_train,
  epochs=EPOCHS, validation_split = 0.2, verbose=1)

Epoch 1/2500
31/31 [==============================] - 0s 9ms/step - loss: 24.7831 - laplace_log_metric: -24.7830 - mae: 2708.3650 - mse: 8018605.0000 - val_loss: 24.7981 - val_laplace_log_metric: -24.7981 - val_mae: 2650.5276 - val_mse: 7761819.0000
Epoch 2/2500
31/31 [==============================] - 0s 2ms/step - loss: 24.7829 - laplace_log_metric: -24.7829 - mae: 2708.1482 - mse: 8017594.5000 - val_loss: 24.7981 - val_laplace_log_metric: -24.7981 - val_mae: 2650.2251 - val_mse: 7760447.0000
Epoch 3/2500
31/31 [==============================] - 0s 2ms/step - loss: 24.7828 - laplace_log_metric: -24.7829 - mae: 2707.8154 - mse: 8016035.5000 - val_loss: 24.7981 - val_laplace_log_metric: -24.7981 - val_mae: 2649.9458 - val_mse: 7759191.5000
Epoch 4/2500
31/31 [==============================] - 0s 2ms/step - loss: 24.7827 - laplace_log_metric: -24.7828 - mae: 2707.5330 - mse: 8014738.5000 - val_loss: 24.7981 - val_laplace_log_metric: -24.7981 - val_mae: 2649.6675 - val_mse: 7757940.0000


Epoch 34/2500
31/31 [==============================] - 0s 2ms/step - loss: 20.4469 - laplace_log_metric: -20.4476 - mae: 2632.0874 - mse: 7718664.5000 - val_loss: 18.5690 - val_laplace_log_metric: -18.5307 - val_mae: 2566.0625 - val_mse: 7449646.5000
Epoch 35/2500
31/31 [==============================] - 0s 3ms/step - loss: 18.1020 - laplace_log_metric: -18.1004 - mae: 2622.1038 - mse: 7686030.0000 - val_loss: 16.6981 - val_laplace_log_metric: -16.6652 - val_mae: 2554.8271 - val_mse: 7415723.0000
Epoch 36/2500
31/31 [==============================] - 0s 2ms/step - loss: 16.4433 - laplace_log_metric: -16.4397 - mae: 2611.0723 - mse: 7650804.0000 - val_loss: 15.2074 - val_laplace_log_metric: -15.1788 - val_mae: 2542.8047 - val_mse: 7381095.0000
Epoch 37/2500
31/31 [==============================] - 0s 2ms/step - loss: 15.0650 - laplace_log_metric: -15.0629 - mae: 2598.9922 - mse: 7613807.0000 - val_loss: 13.9915 - val_laplace_log_metric: -13.9665 - val_mae: 2529.9548 - val_mse: 7346368.0

Epoch 67/2500
31/31 [==============================] - 0s 3ms/step - loss: 8.5243 - laplace_log_metric: -8.5243 - mae: 2523.4009 - mse: 8095917.5000 - val_loss: 8.5144 - val_laplace_log_metric: -8.5113 - val_mae: 2498.2424 - val_mse: 8070275.0000
Epoch 68/2500
31/31 [==============================] - 0s 3ms/step - loss: 8.5199 - laplace_log_metric: -8.5200 - mae: 2521.7131 - mse: 8106338.5000 - val_loss: 8.5138 - val_laplace_log_metric: -8.5107 - val_mae: 2496.2974 - val_mse: 8078545.5000
Epoch 69/2500
31/31 [==============================] - 0s 3ms/step - loss: 8.5127 - laplace_log_metric: -8.5127 - mae: 2518.4146 - mse: 8107712.5000 - val_loss: 8.5132 - val_laplace_log_metric: -8.5101 - val_mae: 2494.3853 - val_mse: 8085134.5000
Epoch 70/2500
31/31 [==============================] - 0s 3ms/step - loss: 8.5128 - laplace_log_metric: -8.5127 - mae: 2516.6282 - mse: 8114779.5000 - val_loss: 8.5126 - val_laplace_log_metric: -8.5095 - val_mae: 2491.7043 - val_mse: 8088105.0000
Epoch 71/250

31/31 [==============================] - 0s 3ms/step - loss: 8.4782 - laplace_log_metric: -8.4782 - mae: 2387.2107 - mse: 7985728.5000 - val_loss: 8.4804 - val_laplace_log_metric: -8.4772 - val_mae: 2340.4558 - val_mse: 7870447.5000
Epoch 101/2500
31/31 [==============================] - 0s 3ms/step - loss: 8.4768 - laplace_log_metric: -8.4767 - mae: 2384.1531 - mse: 7986185.0000 - val_loss: 8.4786 - val_laplace_log_metric: -8.4753 - val_mae: 2332.6472 - val_mse: 7846069.5000
Epoch 102/2500
31/31 [==============================] - 0s 3ms/step - loss: 8.4732 - laplace_log_metric: -8.4731 - mae: 2374.0090 - mse: 7942320.5000 - val_loss: 8.4762 - val_laplace_log_metric: -8.4729 - val_mae: 2322.6863 - val_mse: 7807733.0000
Epoch 103/2500
31/31 [==============================] - 0s 3ms/step - loss: 8.4734 - laplace_log_metric: -8.4735 - mae: 2366.1274 - mse: 7911859.5000 - val_loss: 8.4744 - val_laplace_log_metric: -8.4712 - val_mae: 2317.7476 - val_mse: 7809487.0000
Epoch 104/2500
31/31 [=

31/31 [==============================] - 0s 3ms/step - loss: 8.3535 - laplace_log_metric: -8.3536 - mae: 1800.6537 - mse: 5298651.5000 - val_loss: 8.3276 - val_laplace_log_metric: -8.3268 - val_mae: 1734.6615 - val_mse: 5153059.5000
Epoch 134/2500
31/31 [==============================] - 0s 2ms/step - loss: 8.3511 - laplace_log_metric: -8.3511 - mae: 1786.1451 - mse: 5235773.0000 - val_loss: 8.3244 - val_laplace_log_metric: -8.3237 - val_mae: 1721.5679 - val_mse: 5099049.5000
Epoch 135/2500
31/31 [==============================] - 0s 2ms/step - loss: 8.3459 - laplace_log_metric: -8.3460 - mae: 1772.0791 - mse: 5176853.0000 - val_loss: 8.3205 - val_laplace_log_metric: -8.3199 - val_mae: 1707.0219 - val_mse: 5034896.5000
Epoch 136/2500
31/31 [==============================] - 0s 2ms/step - loss: 8.3453 - laplace_log_metric: -8.3453 - mae: 1759.1775 - mse: 5119318.5000 - val_loss: 8.3173 - val_laplace_log_metric: -8.3167 - val_mae: 1694.3923 - val_mse: 4980643.5000
Epoch 137/2500
31/31 [=

...

31/31 [==============================] - 0s 2ms/step - loss: 7.1581 - laplace_log_metric: -7.1581 - mae: 191.1463 - mse: 158037.8594 - val_loss: 7.2074 - val_laplace_log_metric: -7.2103 - val_mae: 228.7125 - val_mse: 210535.5156
Epoch 2358/2500
31/31 [==============================] - 0s 2ms/step - loss: 7.1511 - laplace_log_metric: -7.1511 - mae: 191.1691 - mse: 157816.9375 - val_loss: 7.2028 - val_laplace_log_metric: -7.2058 - val_mae: 229.9870 - val_mse: 217858.9688
Epoch 2359/2500
31/31 [==============================] - 0s 2ms/step - loss: 7.1597 - laplace_log_metric: -7.1597 - mae: 192.7308 - mse: 161841.7656 - val_loss: 7.2060 - val_laplace_log_metric: -7.2088 - val_mae: 222.5929 - val_mse: 197223.0156
Epoch 2360/2500
31/31 [==============================] - 0s 3ms/step - loss: 7.1583 - laplace_log_metric: -7.1583 - mae: 190.9134 - mse: 156365.9844 - val_loss: 7.2069 - val_laplace_log_metric: -7.2097 - val_mae: 223.4384 - val_mse: 199508.2188
Epoch 2361/2500
31/31 [=============

31/31 [==============================] - 0s 2ms/step - loss: 7.1568 - laplace_log_metric: -7.1567 - mae: 190.4657 - mse: 159742.2656 - val_loss: 7.2051 - val_laplace_log_metric: -7.2080 - val_mae: 230.1818 - val_mse: 215665.0000
Epoch 2391/2500
31/31 [==============================] - 0s 2ms/step - loss: 7.1604 - laplace_log_metric: -7.1603 - mae: 191.6091 - mse: 157064.1562 - val_loss: 7.2033 - val_laplace_log_metric: -7.2062 - val_mae: 228.3523 - val_mse: 212273.8594
Epoch 2392/2500
31/31 [==============================] - 0s 2ms/step - loss: 7.1605 - laplace_log_metric: -7.1604 - mae: 190.2902 - mse: 155872.3906 - val_loss: 7.2035 - val_laplace_log_metric: -7.2064 - val_mae: 226.3112 - val_mse: 206854.5781
Epoch 2393/2500
31/31 [==============================] - 0s 2ms/step - loss: 7.1605 - laplace_log_metric: -7.1603 - mae: 191.8065 - mse: 157759.4219 - val_loss: 7.2084 - val_laplace_log_metric: -7.2112 - val_mae: 226.0255 - val_mse: 204291.9375
Epoch 2394/2500
31/31 [=============

31/31 [==============================] - 0s 2ms/step - loss: 7.1578 - laplace_log_metric: -7.1580 - mae: 192.3782 - mse: 162473.3125 - val_loss: 7.1994 - val_laplace_log_metric: -7.2024 - val_mae: 231.3488 - val_mse: 224713.6094
Epoch 2424/2500
31/31 [==============================] - 0s 2ms/step - loss: 7.1568 - laplace_log_metric: -7.1569 - mae: 192.0397 - mse: 160688.5469 - val_loss: 7.2025 - val_laplace_log_metric: -7.2055 - val_mae: 228.1009 - val_mse: 213082.7344
Epoch 2425/2500
31/31 [==============================] - 0s 2ms/step - loss: 7.1538 - laplace_log_metric: -7.1538 - mae: 191.1672 - mse: 158897.4375 - val_loss: 7.2026 - val_laplace_log_metric: -7.2056 - val_mae: 229.1150 - val_mse: 215014.0156
Epoch 2426/2500
31/31 [==============================] - 0s 2ms/step - loss: 7.1588 - laplace_log_metric: -7.1589 - mae: 190.5599 - mse: 158780.7500 - val_loss: 7.2064 - val_laplace_log_metric: -7.2093 - val_mae: 225.4458 - val_mse: 205025.9844
Epoch 2427/2500
31/31 [=============

31/31 [==============================] - 0s 2ms/step - loss: 7.1573 - laplace_log_metric: -7.1572 - mae: 190.9462 - mse: 158882.5625 - val_loss: 7.2013 - val_laplace_log_metric: -7.2042 - val_mae: 223.7739 - val_mse: 204048.0625
Epoch 2457/2500
31/31 [==============================] - 0s 2ms/step - loss: 7.1598 - laplace_log_metric: -7.1598 - mae: 190.1378 - mse: 155330.4688 - val_loss: 7.2017 - val_laplace_log_metric: -7.2047 - val_mae: 229.0246 - val_mse: 214581.4062
Epoch 2458/2500
31/31 [==============================] - 0s 2ms/step - loss: 7.1610 - laplace_log_metric: -7.1611 - mae: 192.2996 - mse: 160973.0781 - val_loss: 7.2015 - val_laplace_log_metric: -7.2045 - val_mae: 225.6786 - val_mse: 207791.0625
Epoch 2459/2500
31/31 [==============================] - 0s 2ms/step - loss: 7.1591 - laplace_log_metric: -7.1591 - mae: 191.8052 - mse: 160976.8125 - val_loss: 7.2038 - val_laplace_log_metric: -7.2068 - val_mae: 231.1984 - val_mse: 221039.5312
Epoch 2460/2500
31/31 [=============

31/31 [==============================] - 0s 2ms/step - loss: 7.1571 - laplace_log_metric: -7.1571 - mae: 191.6562 - mse: 160723.7969 - val_loss: 7.2020 - val_laplace_log_metric: -7.2049 - val_mae: 225.8443 - val_mse: 209414.2188
Epoch 2490/2500
31/31 [==============================] - 0s 2ms/step - loss: 7.1553 - laplace_log_metric: -7.1550 - mae: 191.0917 - mse: 160837.9219 - val_loss: 7.2048 - val_laplace_log_metric: -7.2078 - val_mae: 230.5988 - val_mse: 217271.6094
Epoch 2491/2500
31/31 [==============================] - 0s 3ms/step - loss: 7.1629 - laplace_log_metric: -7.1631 - mae: 191.5320 - mse: 161115.3281 - val_loss: 7.2034 - val_laplace_log_metric: -7.2063 - val_mae: 228.0165 - val_mse: 213083.8750
Epoch 2492/2500
31/31 [==============================] - 0s 2ms/step - loss: 7.1500 - laplace_log_metric: -7.1501 - mae: 190.4164 - mse: 156288.0156 - val_loss: 7.2069 - val_laplace_log_metric: -7.2096 - val_mae: 220.7067 - val_mse: 194700.8906
Epoch 2493/2500
31/31 [=============

In [63]:
def inference(model, X_test, y_test):
    y_pred = model.predict(X_test).reshape(-1)
    return y_pred, laplace_log_likelihood(y_test, y_pred, np.std(y_pred))

y_pred, metric = inference(model, normalized_X_test, y_test.tolist())
print(metric)

-7.242855437002633


In [64]:
np.std(y_pred)

681.1655

In [65]:
loss, log_laplace, mae, mse = model.evaluate(normalized_X_test, y_test, verbose=2)

10/10 - 0s - loss: 7.2160 - laplace_log_metric: -7.2124e+00 - mae: 192.4710 - mse: 105196.7109
